In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
import os
os.chdir('/content/gdrive/My Drive/2021 2학기/자연어처리(정윤경)')

# 1. 글자 단위 RNN 언어 모델(Char RNNLM)

이상한나라의 엘리스 다운로드 링크 : http://www.gutenberg.org/files/11/11-0.txt

## 1) 데이터에 대한 이해와 전처리

In [5]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [6]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")
f = open('11-0.txt', 'rb')
sentences = []
for sentence in f: # 데이터를 한 줄씩 읽는다.
    sentence = sentence.strip() # strip()을 통해 \r, \n을 제거한다.
    sentence = sentence.lower() # 소문자화.
    sentence = sentence.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()

In [7]:
sentences[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

In [8]:
total_data = ' '.join(sentences) #하나의 문자열로 통합
print('문자열의 길이 또는 총 글자의 개수: %d' % len(total_data))

문자열의 길이 또는 총 글자의 개수: 159484


In [9]:
print(total_data[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


In [10]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))
# 모든 영어 단어는 총 52개의 알파벳으로 표현 가능합니다.

글자 집합의 크기 : 56


In [11]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [12]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [13]:
seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(total_data) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print ('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2658


In [14]:
train_X = []
train_y = []

for i in range(n_samples):
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 픽한다.
    X_sample = total_data[i * seq_length: (i + 1) * seq_length]

    # 정수 인코딩
    X_encoded = [char_to_index[c] for c in X_sample]
    train_X.append(X_encoded)

    # 오른쪽으로 1칸 쉬프트
    y_sample = total_data[i * seq_length + 1: (i + 1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [15]:
print(train_X[0])

[49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]


In [16]:
print(train_y[0])

[37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]


train_y[0]은 train_X[0]에서 오른쪽으로 한 칸 쉬프트 된 문장임을 알 수 있습니다.

In [17]:
print(train_X[1])

[43, 33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54]


In [18]:
print(train_y[1])

[33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54, 52]


마찬가지로 train_y[1]은 train_X[1]에서 오른쪽으로 한 칸 쉬프트 된 문장임을 알 수 있습니다.

이제 train_X와 train_y에 대해서 원-핫 인코딩을 수행. 임베딩층(embedding layer)을 사용하지 않을 것이므로, 입력 시퀀스인 train_X에 대해서도 원-핫 인코딩을 합니다.

In [19]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [20]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2658, 60, 56)
train_y의 크기(shape) : (2658, 60, 56)


샘플의 수(No. of samples)가 2,658개, 입력 시퀀스의 길이(input_length)가 60, 각 벡터의 차원(input_dim)이 55임. 원-핫 벡터의 차원은 글자 집합의 크기인 56이어야 하므로 원-핫 인코딩이 수행되었음을 알 수 있습니다.

## 2) 모델 설계하기

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [22]:
hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 - 39s - loss: 3.0613 - accuracy: 0.1848 - 39s/epoch - 461ms/step
Epoch 2/80
84/84 - 37s - loss: 2.6766 - accuracy: 0.2620 - 37s/epoch - 438ms/step
Epoch 3/80
84/84 - 35s - loss: 2.3640 - accuracy: 0.3362 - 35s/epoch - 415ms/step
Epoch 4/80
84/84 - 35s - loss: 2.2290 - accuracy: 0.3665 - 35s/epoch - 421ms/step
Epoch 5/80
84/84 - 35s - loss: 2.1286 - accuracy: 0.3917 - 35s/epoch - 421ms/step
Epoch 6/80
84/84 - 35s - loss: 2.0517 - accuracy: 0.4125 - 35s/epoch - 421ms/step
Epoch 7/80
84/84 - 35s - loss: 1.9818 - accuracy: 0.4299 - 35s/epoch - 416ms/step
Epoch 8/80
84/84 - 35s - loss: 1.9210 - accuracy: 0.4467 - 35s/epoch - 417ms/step
Epoch 9/80
84/84 - 35s - loss: 1.8643 - accuracy: 0.4617 - 35s/epoch - 419ms/step
Epoch 10/80
84/84 - 35s - loss: 1.8195 - accuracy: 0.4736 - 35s/epoch - 419ms/step
Epoch 11/80
84/84 - 35s - loss: 1.7743 - accuracy: 0.4855 - 35s/epoch - 421ms/step
Epoch 12/80
84/84 - 36s - loss: 1.7334 - accuracy: 0.4968 - 36s/epoch - 423ms/step
Epoch 13/80
8

In [25]:
import joblib
joblib.dump(model, 'char_RNNLM_model.pkl')

INFO:tensorflow:Assets written to: ram://7304eeab-b2d3-451f-8a1b-6626b05a5c1b/assets


INFO:tensorflow:Assets written to: ram://7304eeab-b2d3-451f-8a1b-6626b05a5c1b/assets


['char_RNNLM_model.pkl']

In [26]:
model = joblib.load('char_RNNLM_model.pkl')

In [27]:
def sentence_generation(model, length):
    # 글자에 대한 랜덤 인덱스 생성
    ix = [np.random.randint(vocab_size)]

    # 랜덤 익덱스로부터 글자 생성
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')

    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [28]:
result = sentence_generation(model, 100)
print(result)

22 번 글자 8 로 예측을 시작!
8 and anyone shooo see said the last wers orly, and un in a hor ous when had all see kepter that the8 and anyone shooo see said the last wers orly, and un in a hor ous when had all see kepter that the 


# 2. 글자 단위 RNN(Char RNN)으로 텍스트 생성하기

다 대 일(many-to-one) 구조의 RNN을 글자 단위로 학습시키고, 텍스트 생성

## 1) 데이터에 대한 이해와 전처리

In [29]:
import numpy as np
from tensorflow.keras.utils import to_categorical

임의로 만든 엉터리 노래가사

In [30]:
raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

위의 텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장

In [31]:
tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [32]:
# 중복을 제거한 글자 집합 생성
char_vocab = sorted(list(set(raw_text)))
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [33]:
vocab_size = len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


In [34]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


여기서는 입력 시퀀스의 길이. 즉, 모든 샘플들의 길이가 10가 되도록 데이터를 구성해보겠습니다. 예측 대상이 되는 글자도 필요하므로 우선 길이가 11이 되도록 데이터를 구성합니다.

In [35]:
length = 11
sequences = []
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


In [36]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [37]:
encoded_sequences = []
for sequence in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    encoded_sequence = [char_to_index[char] for char in sequence] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    encoded_sequences.append(encoded_sequence)

In [38]:
encoded_sequences[:5]

[[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18],
 [0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28],
 [16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17],
 [14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0],
 [28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]]

In [39]:
encoded_sequences = np.array(encoded_sequences)
X_data = encoded_sequences[:,:-1]

# 맨 마지막 위치의 글자를 분리
y_data = encoded_sequences[:,-1]

In [40]:
print(X_data[:5])
print(y_data[:5])

[[ 8  0 16 14 28  0 24 23  0 31]
 [ 0 16 14 28  0 24 23  0 31 18]
 [16 14 28  0 24 23  0 31 18 28]
 [14 28  0 24 23  0 31 18 28 17]
 [28  0 24 23  0 31 18 28 17  0]]
[18 28 17  0 21]


In [41]:
# 원-핫 인코딩
X_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in X_data]
X_data_one_hot = np.array(X_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)

In [42]:
print(X_data_one_hot.shape)

(426, 10, 33)


샘플의 수(No. of samples)가 426개, 입력 시퀀스의 길이(input_length)가 10, 각 벡터의 차원(input_dim)이 33임을 의미합니다. 원-핫 벡터의 차원은 글자 집합의 크기인 33이어야 하므로 X에 대해서 원-핫 인코딩이 수행되었음을 알 수 있습니다.

## 2) 모델 설계하기

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
hidden_units = 64

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(X_data_one_hot.shape[1], X_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

LSTM을 사용하고, 은닉 상태의 크기는 80, 그리고 출력층에 단어 집합의 크기만큼의 뉴런을 배치하여 모델을 설계합니다.

In [45]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_data_one_hot, y_data_one_hot, epochs=100, verbose=2)

Epoch 1/100
14/14 - 2s - loss: 3.4678 - accuracy: 0.1080 - 2s/epoch - 172ms/step
Epoch 2/100
14/14 - 0s - loss: 3.3676 - accuracy: 0.1972 - 86ms/epoch - 6ms/step
Epoch 3/100
14/14 - 0s - loss: 3.1012 - accuracy: 0.1972 - 86ms/epoch - 6ms/step
Epoch 4/100
14/14 - 0s - loss: 2.9952 - accuracy: 0.1972 - 96ms/epoch - 7ms/step
Epoch 5/100
14/14 - 0s - loss: 2.9698 - accuracy: 0.1972 - 93ms/epoch - 7ms/step
Epoch 6/100
14/14 - 0s - loss: 2.9273 - accuracy: 0.1972 - 82ms/epoch - 6ms/step
Epoch 7/100
14/14 - 0s - loss: 2.9168 - accuracy: 0.1972 - 94ms/epoch - 7ms/step
Epoch 8/100
14/14 - 0s - loss: 2.8971 - accuracy: 0.1972 - 85ms/epoch - 6ms/step
Epoch 9/100
14/14 - 0s - loss: 2.8723 - accuracy: 0.1972 - 85ms/epoch - 6ms/step
Epoch 10/100
14/14 - 0s - loss: 2.8533 - accuracy: 0.1995 - 88ms/epoch - 6ms/step
Epoch 11/100
14/14 - 0s - loss: 2.8290 - accuracy: 0.1972 - 95ms/epoch - 7ms/step
Epoch 12/100
14/14 - 0s - loss: 2.7992 - accuracy: 0.1995 - 89ms/epoch - 6ms/step
Epoch 13/100
14/14 - 0s -

In [46]:
import joblib
joblib.dump(model, 'char_RNNLM_model_sent.pkl')

INFO:tensorflow:Assets written to: ram://766aebe2-689d-4a46-8552-f819fe7b2b4c/assets


INFO:tensorflow:Assets written to: ram://766aebe2-689d-4a46-8552-f819fe7b2b4c/assets


['char_RNNLM_model_sent.pkl']

In [47]:
model = joblib.load('char_RNNLM_model_sent.pkl')

In [48]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):

    # 초기 시퀀스
    init_text = seed_text
    sentence = ''

    for _ in range(n):
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))

        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for char, index in char_to_index.items():
            if index == result:
                break

        # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        seed_text = seed_text + char

        # 예측 글자를 문장에 저장
        sentence = sentence + char

    sentence = init_text + sentence
    return sentence

In [49]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80)) #새로 생성된 문장

I get on with life as a programmer, I like to contemplate beer. But when I stop my talking
